In [ ]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy import stats
from pprint import pprint

In [ ]:
# Make a reference to the comic_books_expanded.csv file path
income = "US Counties Education vs. Income.csv"
costindex = "data.csv"

In [ ]:
# Import the comic_books_expanded.csv file as a DataFrame
income_df = pd.read_csv(income)
costindex_df = pd.read_csv(costindex)

In [ ]:
# Check the special characters imported correctly
income_df.head()

In [ ]:
costindex_df.head()

In [ ]:
# Remove unecessary columns from the DataFrame and save the new DataFrame
# Only keep: "state","costIndex" 

cost_reduced_df =costindex_df[["state", "costIndex" ]]
cost_reduced_df.head()

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
# Change state to abbreviation
new_cost_df =cost_reduced_df.replace({"state": us_state_to_abbrev})
new_cost_df.head()

In [ ]:
# Merge the two data sets
merged_df = pd.merge(income_df, new_cost_df, on=["state"])
merged_df.head()

In [ ]:
# Calculate the adjusted per capita income
merged_df["Adjusted per capita income 2021"] = merged_df['per_capita_personal_income_2021']/merged_df['costIndex']*100
merged_df

In [ ]:
# Then Format to go to two decimal places, include a dollar sign, and use comma notation
merged_df['Adjusted per capita income 2021'] = merged_df['Adjusted per capita income 2021'].astype('float64').map("${:,.2f}".format)
merged_df

In [ ]:
# Rename column bachelor_degree_percentage_2015_2019 to bachelor_degree_percentage_2016_2019
renamed_df = merged_df.rename(columns={"bachelor_degree_percentage_2015_2019": "bachelor_degree_percentage_2016_2020"})
renamed_df

In [ ]:
# Remove columns that not needed
final_df = renamed_df[["state","county","per_capita_personal_income_2021","associate_degree_numbers_2016_2020",\
                     "bachelor_degree_numbers_2016_2020","associate_degree_percentage_2016_2020","bachelor_degree_percentage_2016_2020",\
                     "costIndex","Adjusted per capita income 2021"]].copy()
final_df.head()

In [ ]:
# Creating a dictionary to rename columns
col_dict = {"per_capita_personal_income_2021":"Per Capita Income 2021",
               "associate_degree_numbers_2016_2020":"Count of Associate Degrees",
               "bachelor_degree_numbers_2016_2020":"Count of Bachelor Degrees",
               "associate_degree_percentage_2016_2020":"Percent of Associate Degrees",
               "bachelor_degree_percentage_2016_2020":"Percent of Bachelor Degrees"
              }

In [ ]:
final_df.rename(columns = col_dict, inplace=True)

In [ ]:
# Using the group by to find the averages by state
state_avgs = final_df.groupby(["state"]).mean()
state_avgs.head()

In [ ]:
# finding correlation coefficient for Associate degree
(slope, interecept,rvalue,pvalue,stderr) = stats.linregress(state_avgs["Percent of Associate Degrees"],state_avgs["Adjusted Per Capita Personal Income 2021"])
y_pred = slope*state_avgs["Percent of Associate Degrees"]+interecept

In [ ]:
plt.scatter(state_avgs["Percent of Associate Degrees"],state_avgs["Adjusted Per Capita Personal Income 2021"])
plt.plot(state_avgs["Percent of Associate Degrees"],y_pred,color="red")
print(f'R-Squared is {rvalue**2}')
plt.show()

In [ ]:
# Finding correlation coefficient for Bachelor degree
(slope, interecept,rvalue,pvalue,stderr) = stats.linregress(state_avgs["Percent of Bachelor Degrees"],state_avgs["Adjusted Per Capita Personal Income 2021"])
y_pred = slope*state_avgs["Percent of Bachelor Degrees"]+interecept

In [ ]:
plt.scatter(state_avgs["Percent of Bachelor Degrees"],state_avgs["Adjusted Per Capita Personal Income 2021"])
print(f'R-Squared is {rvalue**2}')
plt.show()

Split states into regions

In [ ]:
# Use the Pandas copy function to create DataFrame called region_df to store region
region_df= final_df.copy()
# Add an empty column, "region," to the DataFrame so you can store the region the state belong
region_df['region']= ""
# Display sample data
region_df

In [ ]:
# Add region to the DataFrame

South= ['WV', 'DC', 'MD', 'VA', 'KY', 'TN', 'NC', 'MS', 'AR', 'LA', 'AL', 'GA', 'SC', 'FL', 'DE']   
Southwest= ['AZ', 'NM', 'OK', 'TX']    
West= ['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AK', 'HI']    
Midwest= ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'MI', 'IN', 'OH']    
Northeast= ['ME', 'VT', 'NY', 'NH', 'MA', 'RI', 'CT', 'NJ', 'PA']


region_df['region'] = np.where(region_df['state'].isin(South), 'South', np. NaN)
region_df['region'] = np.where(region_df['state'].isin(Southwest), 'Southwest', region_df['region'])                               
region_df['region'] = np.where(region_df['state'].isin(West), 'West', region_df['region'])                               
region_df['region'] = np.where(region_df['state'].isin(Midwest), 'Midwest', region_df['region'] )                              
region_df['region'] = np.where(region_df['state'].isin(Northeast), 'Northeast', region_df['region'])


region_df.head()    
        
      


In [ ]:
# Split DataFrame into regions
south_region_df = region_df.loc[region_df["region"]== "South", :]

# Display sample data
south_region_df.head()

In [ ]:
southwest_region_df = region_df.loc[region_df["region"]== "Southwest", :]
southwest_region_df.head()

In [ ]:
west_region_df = region_df.loc[region_df["region"]== "West", :]
west_region_df.head()

In [ ]:
midwest_region_df = region_df.loc[region_df["region"]== "Midwest", :]
midwest_region_df.head()

In [ ]:
northeast_region_df = region_df.loc[region_df["region"]== "Northeast", :]
northeast_region_df.head()